In [62]:
import yfinance as yf
from prophet import Prophet
import pandas as pd
from datetime import datetime, timedelta

In [63]:
def get_marketcap(ticker):
    try:
        yfin = yf.Ticker(ticker)
        info = yfin.info
        market_cap = info.get('marketCap', None)
        return market_cap
    except Exception as e:
        print(f"Error getting market cap for {ticker}: {e}")
        return None

def classify_market_cap(market_cap):
    if market_cap is not None:
        try:
            market_cap = float(market_cap)
            if market_cap >= 900000000000:
                return 'largecap'
            elif 90000000000 <= market_cap < 900000000000:
                return 'midcap'
            else:
                return 'smallcap'
        except ValueError as e:
            print(f"Error classifying market cap: {e}")
            return None
    return None

def get_short_name(ticker):
    try:
        yfin = yf.Ticker(ticker)
        info = yfin.info
        return info.get('shortName', None)
    except Exception as e:
        print(f"Error getting short name for {ticker}: {e}")
        return None

def get_actual_stock_price(ticker, date):
    try:
        yfin = yf.Ticker(ticker)
        hist = yfin.history(start="2024-02-08", end="2024-02-09")
        if not hist.empty:
            return hist['Close'].iloc[0]
        return None
    except Exception as e:
        print(f"Error getting actual stock price for {ticker}: {e}")
        return None

def get_fundamentals(ticker):
    try:
        yfin = yf.Ticker(ticker)
        info = yfin.info
        return {
            'sector': info.get('sector', None),
            'recommendationKey': info.get('recommendationKey', None),
            'numberOfAnalystOpinions': info.get('numberOfAnalystOpinions', None),
            'totalCash': info.get('totalCash', None),
            'totalDebt': info.get('totalDebt', None),
            'totalRevenue': info.get('totalRevenue', None),
            'returnOnEquity': info.get('returnOnEquity', None),
        }
    except Exception as e:
        print(f"Error getting fundamentals for {ticker}: {e}")
        return {
            'sector': None,
            'recommendationKey': None,
            'numberOfAnalystOpinions': None,
            'totalCash': None,
            'totalDebt': None,
            'totalRevenue': None,
            'returnOnEquity': None,
        }

def predict(ticker, days):
    try:
        yfin = yf.Ticker(ticker)
        hist = yfin.history(period="max")
        hist = hist[['Close']]
        hist.reset_index(level=0, inplace=True)
        hist = hist.rename({'Date': 'ds', 'Close': 'y'}, axis='columns')
        m = Prophet(daily_seasonality=True)
        hist['ds'] = hist['ds'].dt.tz_localize(None)
        m.fit(hist)
        future = m.make_future_dataframe(periods=days)
        forecast = m.predict(future)
        return forecast
    except Exception as e:
        print(f"Error predicting for {ticker}: {e}")
        return None

In [64]:
# Set the date for which you want to get the actual stock price
tday_date = "2024-02-09"

# List of ticker symbols for largecap, midcap, and smallcap stocks
largecap_tickers = [
    # Large-cap stocks
    'RELIANCE.BO', 'TCS.BO', 'HDFCBANK.BO', 'HINDUNILVR.BO', 'INFY.BO', 'KOTAKBANK.BO', 'ITC.BO', 'ICICIBANK.BO', 'SBIN.BO', 'ASIANPAINT.BO',
    'HDFCLIFE.BO', 'BAJAJFINSV.BO', 'BAJFINANCE.BO', 'LT.BO', 'ONGC.BO', 'MARUTI.BO', 'SUNPHARMA.BO', 'AXISBANK.BO', 'BHARTIARTL.BO', 'ULTRACEMCO.BO',
    'NTPC.BO', 'HCLTECH.BO', 'WIPRO.BO', 'POWERGRID.BO', 'NESTLEIND.BO', 'JSWSTEEL.BO', 'GRASIM.BO', 'BAJAJ-AUTO.BO', 'SHREECEM.BO',
    'HINDZINC.BO', 'DRREDDY.BO', 'DIVISLAB.BO', 'TITAN.BO', 'HEROMOTOCO.BO', 'COALINDIA.BO', 'IOC.BO', 'IOC.BO', 'BIOCON.BO', 'DABUR.BO',
    'INDUSINDBK.BO', 'PIDILITIND.BO', 'ICICIPRULI.BO', 'UBL.BO', 'MUTHOOTFIN.BO', 'BERGEPAINT.BO', 'PGHH.BO', 'DLF.BO', 'CUMMINSIND.BO',
    'MRF.BO',

    # Mid-cap stocks
    'SIEMENS.BO', 'RAMCOCEM.BO', 'CROMPTON.BO', 'PNB.BO', 'MINDTREE.BO', 'GODREJPROP.BO', 'ATUL.BO', 'TVSMOTOR.BO', 'BANKINDIA.BO', 'IDBI.BO',
    'BALKRISIND.BO', 'APOLLOHOSP.BO', 'AJANTPHARM.BO', 'BEL.BO', 'SYNGENE.BO', 'CUB.BO', 'FEDERALBNK.BO', 'CASTROLIND.BO', 'MGL.BO', 'MCDOWELL-N.BO',
    'GUJGASLTD.BO', 'VBL.BO', 'ENDURANCE.BO', 'ISEC.BO', 'LALPATHLAB.BO', 'ISEC.BO', 'SIS.BO', 'ISEC.BO', 'AMBER.BO', 'VGUARD.BO',
    'AARTIIND.BO', 'APLLTD.BO', 'VAKRANGEE.BO', 'LTI.BO', 'NESCO.BO', 'GESHIP.BO', 'MRPL.BO', 'FORTIS.BO', 'IDFCFIRSTB.BO', 'ADANIGREEN.BO',
    'STAR.BO', 'ASTRAL.BO', 'ESCORTS.BO', 'FRETAIL.BO', 'CDSL.BO', 'JUBLFOOD.BO', 'CHOLAFIN.BO',

    # Small-cap stocks
    'HSCL.BO', 'SIS.BO', 'ASHIANA.BO', 'J&KBANK.BO', 'SUPRAJIT.BO', 'INDOCO.BO', 'HIMATSEIDE.BO', 'ARVIND.BO', 'JKCEMENT.BO', 'KANSAINER.BO',
    'EDELWEISS.BO', 'RAMASTEEL.BO', 'KPITTECH.BO', 'ASTRAMICRO.BO', 'ISEC.BO', 'SHARDACROP.BO', 'SPARC.BO', 'ARVINDFASN.BO', 'RELAXO.BO', 'PHILIPCARB.BO',
    'NIITLTD.BO', 'IRCON.BO', 'EMAMILTD.BO', 'GSPL.BO', 'TEAMLEASE.BO', 'GREAVESCOT.BO', 'AMARAJABAT.BO', 'PRSMJOHNSN.BO', 'JCHAC.BO', 'SARDAEN.BO',
    'APARINDS.BO', 'APLAPOLLO.BO', 'FINEORG.BO', 'TCIEXP.BO', 'MAHSCOOTER.BO', 'ESCORTS.BO', 'NBCC.BO', 'TATASTLBSL.BO', 'SUMICHEM.BO', 'PGIL.BO',
    'MIDHANI.BO', 'DHANI.BO', 'JKPAPER.BO', 'TATAMETALI.BO', 'BASF.BO', 'BLUESTARCO.BO', 'LAURUSLABS.BO', '3MINDIA.BO', 'LINDEINDIA.BO'
]

# Initialize an empty DataFrame to store the results
all_data = pd.DataFrame()

# Iterate through largecap tickers
for ticker in largecap_tickers:
    # Get actual stock price on the specified date
    tday_stock_price = get_actual_stock_price(ticker, tday_date)

    # Skip if actual stock price is not available
    if tday_stock_price is None:
        continue

    # Get shortName and marketCap
    short_name = get_short_name(ticker)
    market_cap = get_marketcap(ticker)

    # Classify market cap
    market_cap_classification = classify_market_cap(market_cap)

    # Get fundamentals
    fundamentals = get_fundamentals(ticker)

    # Get predictions
    forecast = predict(ticker, 365)

    # Skip if prediction is not available
    if forecast is None:
        continue

    # Extract predicted values for 3 months, 6 months, and 12 months after tday
    tday_index = forecast[forecast['ds'] == tday_date].index[0]
    pred_3_months = forecast['yhat'].iloc[tday_index + 90]
    pred_6_months = forecast['yhat'].iloc[tday_index + 180]
    pred_12_months = forecast['yhat'].iloc[tday_index + 365]

    # Calculate percentage gain
    percentage_gain = ((pred_12_months - tday_stock_price) / tday_stock_price) * 100

    # Create a DataFrame for the current stock
    data = {
        'shortName': [short_name],
        'marketCap': [market_cap_classification],
        'actual_price_on_tday': [tday_stock_price],
        'pred_3_months': [pred_3_months],
        'pred_6_months': [pred_6_months],
        'pred_12_months': [pred_12_months],
        'percentage_gain': [percentage_gain],
        'sector': [fundamentals['sector']],
        'recommendationKey': [fundamentals['recommendationKey']],
        'numberOfAnalystOpinions': [fundamentals['numberOfAnalystOpinions']],
        'totalCash': [fundamentals['totalCash']],
        'totalDebt': [fundamentals['totalDebt']],
        'totalRevenue': [fundamentals['totalRevenue']],
        'returnOnEquity': [fundamentals['returnOnEquity']],
    }

    stock_data = pd.DataFrame(data)

    # Append to the overall DataFrame
    all_data = all_data.append(stock_data, ignore_index=True)

# Save the overall DataFrame to CSV
all_data.to_csv('stock_analysis.csv', index=False)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpjyvr7n6r/oym6ojdm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjyvr7n6r/fuxark31.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29981', 'data', 'file=/tmp/tmpjyvr7n6r/oym6ojdm.json', 'init=/tmp/tmpjyvr7n6r/fuxark31.json', 'output', 'file=/tmp/tmpjyvr7n6r/prophet_modela1jzl_bq/prophet_model-20240211015612.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:56:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:56:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-64-0a6a9edfb3fc>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(stock_data, ignore_index=True)
DEBUG:cmdstan

In [61]:
import yfinance as yf
# List of ticker symbols for largecap, midcap, and smallcap stocks
largecap_tickers = [
    # Large-cap stocks
    'RELIANCE.BO', 'TCS.BO', 'HDFCBANK.BO', 'HINDUNILVR.BO', 'INFY.BO', 'KOTAKBANK.BO', 'ITC.BO', 'ICICIBANK.BO', 'SBIN.BO', 'ASIANPAINT.BO',
    'HDFCLIFE.BO', 'BAJAJFINSV.BO', 'BAJFINANCE.BO', 'LT.BO', 'ONGC.BO', 'MARUTI.BO', 'SUNPHARMA.BO', 'AXISBANK.BO', 'BHARTIARTL.BO', 'ULTRACEMCO.BO',
    'NTPC.BO', 'HCLTECH.BO', 'WIPRO.BO', 'POWERGRID.BO', 'NESTLEIND.BO', 'JSWSTEEL.BO', 'GRASIM.BO', 'BAJAJ-AUTO.BO', 'SHREECEM.BO',
    'HINDZINC.BO', 'DRREDDY.BO', 'DIVISLAB.BO', 'TITAN.BO', 'HEROMOTOCO.BO', 'COALINDIA.BO', 'IOC.BO', 'IOC.BO', 'BIOCON.BO', 'DABUR.BO',
    'INDUSINDBK.BO', 'PIDILITIND.BO', 'ICICIPRULI.BO', 'UBL.BO', 'MUTHOOTFIN.BO', 'BERGEPAINT.BO', 'PGHH.BO', 'DLF.BO', 'CUMMINSIND.BO',
    'MRF.BO',

    # Mid-cap stocks
    'SIEMENS.BO', 'RAMCOCEM.BO', 'CROMPTON.BO', 'PNB.BO', 'MINDTREE.BO', 'GODREJPROP.BO', 'ATUL.BO', 'TVSMOTOR.BO', 'BANKINDIA.BO', 'IDBI.BO',
    'BALKRISIND.BO', 'APOLLOHOSP.BO', 'AJANTPHARM.BO', 'BEL.BO', 'SYNGENE.BO', 'CUB.BO', 'FEDERALBNK.BO', 'CASTROLIND.BO', 'MGL.BO', 'MCDOWELL-N.BO',
    'GUJGASLTD.BO', 'VBL.BO', 'ENDURANCE.BO', 'ISEC.BO', 'LALPATHLAB.BO', 'ISEC.BO', 'SIS.BO', 'ISEC.BO', 'AMBER.BO', 'VGUARD.BO',
    'AARTIIND.BO', 'APLLTD.BO', 'VAKRANGEE.BO', 'LTI.BO', 'NESCO.BO', 'GESHIP.BO', 'MRPL.BO', 'FORTIS.BO', 'IDFCFIRSTB.BO', 'ADANIGREEN.BO',
    'STAR.BO', 'ASTRAL.BO', 'ESCORTS.BO', 'FRETAIL.BO', 'CDSL.BO', 'JUBLFOOD.BO', 'CHOLAFIN.BO',

    # Small-cap stocks
    'HSCL.BO', 'SIS.BO', 'ASHIANA.BO', 'J&KBANK.BO', 'SUPRAJIT.BO', 'INDOCO.BO', 'HIMATSEIDE.BO', 'ARVIND.BO', 'JKCEMENT.BO', 'KANSAINER.BO',
    'EDELWEISS.BO', 'RAMASTEEL.BO', 'KPITTECH.BO', 'ASTRAMICRO.BO', 'ISEC.BO', 'SHARDACROP.BO', 'SPARC.BO', 'ARVINDFASN.BO', 'RELAXO.BO', 'PHILIPCARB.BO',
    'NIITLTD.BO', 'IRCON.BO', 'EMAMILTD.BO', 'GSPL.BO', 'TEAMLEASE.BO', 'GREAVESCOT.BO', 'AMARAJABAT.BO', 'PRSMJOHNSN.BO', 'JCHAC.BO', 'SARDAEN.BO',
    'APARINDS.BO', 'APLAPOLLO.BO', 'FINEORG.BO', 'TCIEXP.BO', 'MAHSCOOTER.BO', 'ESCORTS.BO', 'NBCC.BO', 'TATASTLBSL.BO', 'SUMICHEM.BO', 'PGIL.BO',
    'MIDHANI.BO', 'DHANI.BO', 'JKPAPER.BO', 'TATAMETALI.BO', 'BASF.BO', 'BLUESTARCO.BO', 'LAURUSLABS.BO', '3MINDIA.BO', 'LINDEINDIA.BO'
]

for stock_symbol in largecap_tickers:
    market_cap = get_marketcap(stock_symbol)
    print(market_cap)
    # Classify market cap
    market_cap_classification = classify_market_cap(market_cap)
    print("For : ", stock_symbol, " marketcap is ", market_cap_classification)


19772603367424
For :  RELIANCE.BO  marketcap is  largecap
14958088683520
For :  TCS.BO  marketcap is  largecap
10661120704512
For :  HDFCBANK.BO  marketcap is  largecap
5695875842048
For :  HINDUNILVR.BO  marketcap is  largecap
6911015518208
For :  INFY.BO  marketcap is  largecap
3463446528000
For :  KOTAKBANK.BO  marketcap is  largecap
5186314043392
For :  ITC.BO  marketcap is  largecap
7092129234944
For :  ICICIBANK.BO  marketcap is  largecap
6472571813888
For :  SBIN.BO  marketcap is  largecap
2830645329920
For :  ASIANPAINT.BO  marketcap is  largecap
1255560183808
For :  HDFCLIFE.BO  marketcap is  largecap
2501427855360
For :  BAJAJFINSV.BO  marketcap is  largecap
4117151612928
For :  BAJFINANCE.BO  marketcap is  largecap
4570576060416
For :  LT.BO  marketcap is  largecap
3358311055360
For :  ONGC.BO  marketcap is  largecap
3374437367808
For :  MARUTI.BO  marketcap is  largecap
3682611757056
For :  SUNPHARMA.BO  marketcap is  largecap
3242352705536
For :  AXISBANK.BO  marketcap is 

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LTI.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LTI.BO&crumb=%2FXfjqhr5SJ0


None
For :  LTI.BO  marketcap is  None
60112949248
For :  NESCO.BO  marketcap is  smallcap
137106038784
For :  GESHIP.BO  marketcap is  midcap
331942428672
For :  MRPL.BO  marketcap is  midcap
327916027904
For :  FORTIS.BO  marketcap is  midcap
574494343168
For :  IDFCFIRSTB.BO  marketcap is  midcap
2977897119744
For :  ADANIGREEN.BO  marketcap is  largecap
64380375040
For :  STAR.BO  marketcap is  smallcap
497618092032
For :  ASTRAL.BO  marketcap is  midcap
304495984640
For :  ESCORTS.BO  marketcap is  midcap
1626851200
For :  FRETAIL.BO  marketcap is  smallcap


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CDSL.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CDSL.BO&crumb=%2FXfjqhr5SJ0


None
For :  CDSL.BO  marketcap is  None
317397139456
For :  JUBLFOOD.BO  marketcap is  midcap
951691051008
For :  CHOLAFIN.BO  marketcap is  largecap
181422735360
For :  HSCL.BO  marketcap is  midcap
67316314112
For :  SIS.BO  marketcap is  smallcap
31348721664
For :  ASHIANA.BO  marketcap is  smallcap
148481540096
For :  J&KBANK.BO  marketcap is  midcap
55878012928
For :  SUPRAJIT.BO  marketcap is  smallcap
32206565376
For :  INDOCO.BO  marketcap is  smallcap
15487318016
For :  HIMATSEIDE.BO  marketcap is  smallcap
76663341056
For :  ARVIND.BO  marketcap is  smallcap
326941245440
For :  JKCEMENT.BO  marketcap is  midcap
256135217152
For :  KANSAINER.BO  marketcap is  midcap
63748780032
For :  EDELWEISS.BO  marketcap is  smallcap
21723955200
For :  RAMASTEEL.BO  marketcap is  smallcap
468463583232
For :  KPITTECH.BO  marketcap is  midcap
61410426880
For :  ASTRAMICRO.BO  marketcap is  smallcap
260062806016
For :  ISEC.BO  marketcap is  midcap
34013126656
For :  SHARDACROP.BO  marketcap

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AMARAJABAT.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AMARAJABAT.BO&crumb=%2FXfjqhr5SJ0


None
For :  AMARAJABAT.BO  marketcap is  None
92139495424
For :  PRSMJOHNSN.BO  marketcap is  midcap
30646861824
For :  JCHAC.BO  marketcap is  smallcap
77752860672
For :  SARDAEN.BO  marketcap is  smallcap
246942662656
For :  APARINDS.BO  marketcap is  midcap
373160083456
For :  APLAPOLLO.BO  marketcap is  midcap
134908592128
For :  FINEORG.BO  marketcap is  midcap
50558771200
For :  TCIEXP.BO  marketcap is  smallcap
86641926144
For :  MAHSCOOTER.BO  marketcap is  smallcap
304495984640
For :  ESCORTS.BO  marketcap is  midcap
266219995136
For :  NBCC.BO  marketcap is  midcap


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TATASTLBSL.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TATASTLBSL.BO&crumb=%2FXfjqhr5SJ0


None
For :  TATASTLBSL.BO  marketcap is  None
191247777792
For :  SUMICHEM.BO  marketcap is  midcap
24396972032
For :  PGIL.BO  marketcap is  smallcap
86345007104
For :  MIDHANI.BO  marketcap is  smallcap
21421039616
For :  DHANI.BO  marketcap is  smallcap
66430992384
For :  JKPAPER.BO  marketcap is  smallcap
35084181504
For :  TATAMETALI.BO  marketcap is  smallcap
139174019072
For :  BASF.BO  marketcap is  midcap
239808790528
For :  BLUESTARCO.BO  marketcap is  midcap
211598065664
For :  LAURUSLABS.BO  marketcap is  midcap
369654661120
For :  3MINDIA.BO  marketcap is  midcap
478649024512
For :  LINDEINDIA.BO  marketcap is  midcap
